# One-vs-All Classification

In a [previous article](), we talked in detail about logistic regression in the case of binary classification; that is, given a set of training data $\{ \, ( \mathbf{x}_1,y_1 ), ( \mathbf{x}_2,y_2 ) \ldots,  ( \mathbf{x}_m,y_m ) \, \}$, logistic regression finds a function $h_\theta(\mathbf{x})$ that estimates the probability of a class label $y \in \{0,1\}$ given a feature vector $\mathbf{x}$.  We can then predict binary outcomes by specifying a probability threshold, and the threshold we use is given as follows

$$ \mathrm{prediction} =
\begin{cases}
1 & \mathrm{if} \quad P(y = 1 \,|\, \mathbf{x} ; \theta) \geq 0.5 \\ 
0 & \mathrm{if} \quad P(y = 0 \,|\, \mathbf{x} ; \theta) < 0.5
\end{cases} 
$$

and recall that $\theta$ is a vector of parameters in the logistic regression model.

One-vs-all classification extends logistic regression to the case of classifying more than two class labels by considering a sequence of binary classifications.  Suppose the predictor variable $y$ can be labeled in $k$ possible ways, so $y \in \{ 0, 1, \ldots, k \}$ with $k \geq 2$.  We can turn this into a sequence of binary classification problems and use logistic regression at each step.  

First separate the labels $\{0, 1, \dots, k \}$ into either 0 or $\{1, \dots, k \}$ and fit a logistic regression model to this.  This logistic regression model will find a function $h_\theta^{(0)}(\mathbf{x})$ that classifies the training data into either 0 or 

In [4]:
import numpy as np  
import pandas as pd 
import os
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

filepath = os.getcwd() + '/logReg_data3.mat'
data = loadmat('logReg_data3.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}